In [1]:
import numpy as np  
import pandas as pd 
import pickle
from prettytable import PrettyTable
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [2]:
saved_dict = {}

In [3]:
# Leer los datasets
datasets = []
for i in range(1,5): # Hay 4 CSV
    path = './UNSW-NB15_{}.csv'  # Importar de los csv
    datasets.append(pd.read_csv(path.format(i), header = None)) # Concatenar datos de csv
dataset = pd.concat(datasets).reset_index(drop=True)  # Guardar todos en un solo dataset


In [4]:
# Importar nombres columnas
columnas_dataset = pd.read_csv('./NUSW-NB15_features.csv', encoding='ISO-8859-1') 

In [5]:
# Todos los nombres en letra pequeña y eliminando espacios
columnas_dataset['Name'] = columnas_dataset['Name'].apply(lambda x: x.strip().replace(' ', '').lower())

In [6]:
# Renombrar las columnas del dataset
dataset.columns = columnas_dataset['Name']

In [7]:
# Saving useful info, later this will be used to transform raw test data
saved_dict['columns'] = columnas_dataset['Name'][columnas_dataset['Name']!='attack_cat'].tolist()

In [8]:
dataset.shape

(2540047, 49)

In [9]:
dataset.head()

Name,srcip,sport,dstip,dsport,proto,state,dur,sbytes,dbytes,sttl,...,ct_ftp_cmd,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,attack_cat,label
0,59.166.0.0,1390,149.171.126.6,53,udp,CON,0.001055,132,164,31,...,0,3,7,1,3,1,1,1,NaN,0
1,59.166.0.0,33661,149.171.126.9,1024,udp,CON,0.036133,528,304,31,...,0,2,4,2,3,1,1,2,NaN,0
2,59.166.0.6,1464,149.171.126.7,53,udp,CON,0.001119,146,178,31,...,0,12,8,1,2,2,1,1,NaN,0
3,59.166.0.5,3593,149.171.126.5,53,udp,CON,0.001209,132,164,31,...,0,6,9,1,1,1,1,1,NaN,0
4,59.166.0.3,49664,149.171.126.0,53,udp,CON,0.001169,146,178,31,...,0,7,9,1,1,1,1,1,NaN,0


In [10]:
# Dividir el dataset en train y test
train, test = train_test_split(dataset, test_size=0.3, random_state=16)

In [11]:
print(train.shape,'\n',test.shape)

(1778032, 49) 
 (762015, 49)


In [12]:
# Revisar si hay valores nulos
train.isnull().sum()

Name
srcip                     0
sport                     0
dstip                     0
dsport                    0
proto                     0
state                     0
dur                       0
sbytes                    0
dbytes                    0
sttl                      0
dttl                      0
sloss                     0
dloss                     0
service                   0
sload                     0
dload                     0
spkts                     0
dpkts                     0
swin                      0
dwin                      0
stcpb                     0
dtcpb                     0
smeansz                   0
dmeansz                   0
trans_depth               0
res_bdy_len               0
sjit                      0
djit                      0
stime                     0
ltime                     0
sintpkt                   0
dintpkt                   0
tcprtt                    0
synack                    0
ackdat                    0
is_sm_ips_ports

In [13]:
train['attack_cat'].value_counts()

Generic             150906
Exploits             31507
 Fuzzers             13345
DoS                  11433
 Reconnaissance       8544
 Fuzzers              3569
Analysis              1855
Backdoor              1242
Reconnaissance        1220
 Shellcode             904
Backdoors              374
Shellcode              151
Worms                  120
Name: attack_cat, dtype: int64

In [14]:
# Sustituir los valores nulos por tráfico "normal"
train['attack_cat'] = train.attack_cat.fillna(value='normal').apply(lambda x: x.strip().lower())
test['attack_cat'] = test.attack_cat.fillna(value='normal').apply(lambda x: x.strip().lower())

In [15]:
train['attack_cat'].value_counts()

normal            1552862
generic            150906
exploits            31507
fuzzers             16914
dos                 11433
reconnaissance       9764
analysis             1855
backdoor             1242
shellcode            1055
backdoors             374
worms                 120
Name: attack_cat, dtype: int64

In [16]:
test['attack_cat'].value_counts()

normal            665902
generic            64575
exploits           13018
fuzzers             7332
dos                 4920
reconnaissance      4223
analysis             822
backdoor             553
shellcode            456
backdoors            160
worms                 54
Name: attack_cat, dtype: int64

In [17]:
# Sustituir los valores nulos por 0
train['ct_flw_http_mthd'] = train.ct_flw_http_mthd.fillna(value=0)

In [18]:
train['is_ftp_login'].value_counts()

0.0    746536
1.0     30329
4.0       109
2.0        21
Name: is_ftp_login, dtype: int64

In [19]:
# Sustituir los valores nulos por 0 con tipo int
train['is_ftp_login'] = (train.is_ftp_login.fillna(value=0)).astype(int)

In [20]:
# Comprobar si sigue exisitiendo algun valor nulo
train.isnull().sum().sum(), test.isnull().sum().sum() 

(0, 833111)

In [21]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1778032 entries, 81030 to 2079982
Data columns (total 49 columns):
 #   Column            Dtype  
---  ------            -----  
 0   srcip             object 
 1   sport             object 
 2   dstip             object 
 3   dsport            object 
 4   proto             object 
 5   state             object 
 6   dur               float64
 7   sbytes            int64  
 8   dbytes            int64  
 9   sttl              int64  
 10  dttl              int64  
 11  sloss             int64  
 12  dloss             int64  
 13  service           object 
 14  sload             float64
 15  dload             float64
 16  spkts             int64  
 17  dpkts             int64  
 18  swin              int64  
 19  dwin              int64  
 20  stcpb             int64  
 21  dtcpb             int64  
 22  smeansz           int64  
 23  dmeansz           int64  
 24  trans_depth       int64  
 25  res_bdy_len       int64  
 26  sjit      

In [22]:
# Tipos de datos
train.dtypes.value_counts()

int64      29
float64    11
object      9
dtype: int64

In [23]:
# Mostrar que valores no son números
train.select_dtypes(exclude=np.number).columns

Index(['srcip', 'sport', 'dstip', 'dsport', 'proto', 'state', 'service',
       'ct_ftp_cmd', 'attack_cat'],
      dtype='object', name='Name')

In [24]:
# Este atributo debería ser numérico
train['ct_ftp_cmd'].unique()

array([0, ' ', 1, '1', '0', 5, 2, 3, 4, '4', 6, '2', 8], dtype=object)

In [25]:
# Reemplazar valores vacíos por 0 para ser numérico
train['ct_ftp_cmd'] = train['ct_ftp_cmd'].replace(to_replace=' ', value=0).astype(int)

In [26]:
# Se comprueba
train.select_dtypes(exclude=np.number).columns

Index(['srcip', 'sport', 'dstip', 'dsport', 'proto', 'state', 'service',
       'attack_cat'],
      dtype='object', name='Name')

In [27]:
# Mostrar valores numéricos
train.select_dtypes(include=np.number).columns

Index(['dur', 'sbytes', 'dbytes', 'sttl', 'dttl', 'sloss', 'dloss', 'sload',
       'dload', 'spkts', 'dpkts', 'swin', 'dwin', 'stcpb', 'dtcpb', 'smeansz',
       'dmeansz', 'trans_depth', 'res_bdy_len', 'sjit', 'djit', 'stime',
       'ltime', 'sintpkt', 'dintpkt', 'tcprtt', 'synack', 'ackdat',
       'is_sm_ips_ports', 'ct_state_ttl', 'ct_flw_http_mthd', 'is_ftp_login',
       'ct_ftp_cmd', 'ct_srv_src', 'ct_srv_dst', 'ct_dst_ltm', 'ct_src_ltm',
       'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'label'],
      dtype='object', name='Name')

In [28]:
# Info for test data transformation
saved_dict['binary_col'] = ['is_sm_ips_ports', 'is_ftp_login']

In [29]:
# Comprobar si las columnas binarias están bien
for col in 'is_sm_ips_ports', 'is_ftp_login':
    print(train[col].value_counts())
    print()

0    1775116
1       2916
Name: is_sm_ips_ports, dtype: int64

0    1747573
1      30329
4        109
2         21
Name: is_ftp_login, dtype: int64



In [30]:
# Convertir a binario
train['is_ftp_login'] = np.where(train['is_ftp_login']>1, 1, train['is_ftp_login'])
train['is_ftp_login'].value_counts()

0    1747573
1      30459
Name: is_ftp_login, dtype: int64

In [31]:
train['service'].value_counts()

-           872286
dns         547094
http        144555
ftp-data     88310
smtp         57160
ftp          34293
ssh          32897
pop3          1076
dhcp           124
ssl            109
snmp            81
radius          27
irc             20
Name: service, dtype: int64

In [32]:
# Reemplazar los valores "-" por "None"
train['service'] = train['service'].apply(lambda x:"None" if x=="-" else x)
train['service'].value_counts()

None        872286
dns         547094
http        144555
ftp-data     88310
smtp         57160
ftp          34293
ssh          32897
pop3          1076
dhcp           124
ssl            109
snmp            81
radius          27
irc             20
Name: service, dtype: int64

In [33]:
train['attack_cat'].value_counts()

normal            1552862
generic            150906
exploits            31507
fuzzers             16914
dos                 11433
reconnaissance       9764
analysis             1855
backdoor             1242
shellcode            1055
backdoors             374
worms                 120
Name: attack_cat, dtype: int64

In [34]:
# Agrupar todos los backdoor
train['attack_cat'] = train['attack_cat'].replace('backdoors','backdoor', regex=True).apply(lambda x: x.strip().lower())
test['attack_cat'] = test['attack_cat'].replace('backdoors','backdoor', regex=True).apply(lambda x: x.strip().lower())

In [35]:
train.shape, test.shape

((1778032, 49), (762015, 49))

In [36]:
train.to_csv('./train_alldata_EDA.csv', index=False)
test.to_csv('./test_alldata_EDA.csv', index=False)
pickle.dump(saved_dict, open('./final_ipynb', 'wb'))